# In this lab, you will understand how to use vector stores and RAG

**STORING DOCUMENTS IN CHROMA**

In [ ]:
pip install langchain-huggingface  langchain_core langchain_openai langchain_chroma langchain_community faiss-cpu pypdf gradio langsmith



**Open .env file in this folder and observe that we have configured OPENAI_API_KEY. Replace it with your own key or key given by me**

The Code in the below cell will load the .env file and set environment variables.

**Write the code in the below cell and execute it**

In [1]:

from dotenv import load_dotenv

load_dotenv()

True

**Creating Documents manually**

 **Document**  represents a unit of text and associated metadata.

 It has two attributes:

**pagecontent** string representing the content

**metadata** a dict containing arbitrary metadata.

In [2]:
from langchain_core.documents import Document



documents = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]


# **Creating  a vector store(Chroma) and Persisting Documents using OpenAIEmbeddings**

**Chroma is a AI-native open-source vector database**

Chroma runs in various modes:

**in-memory**

**in-memory with persistance**  - in a script or notebook and save/load to disk

**in a docker container** -- as a server running your local machine or in the cloud.

Below code creates a vectorstore from the given documents and persists data in given directory.

Execute the below code and observe that a directory with name chromadb1 is created

In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

from langchain_huggingface.embeddings import HuggingFaceEmbeddings

#embeddings = OpenAIEmbeddings()

embeddings= HuggingFaceEmbeddings()

vectorstore = Chroma.from_documents(
    documents, embedding=embeddings , persist_directory="chromadb4"
)

**Let us understand how to create a vector store using the persist_directory**

In [4]:
vectorstore = Chroma(
    persist_directory="chromadb4", embedding_function=embeddings
)

**Using Vector Store as a retriever**

LangChain VectorStore objects do not subclass Runnable
LangChain Retrievers are Runnables. So, we can use them in chains in LCEL

Vectorstores implement an **as_retriever** method that will generate a Retriever, specifically a **VectorStoreRetriever**

Below code shows how to get a retriever from vectorstore object
see the documentation at (https://api.python.langchain.com/en/latest/vectorstores/langchain_community.vectorstores.chroma.Chroma.html#langchain_community.vectorstores.chroma.Chroma.as_retriever)

In [5]:
retriever = vectorstore.as_retriever(
    search_type="similarity",  # mmr or similarity_score_threshold
    search_kwargs={"k": 1},
)

retriever.invoke("tell me about movie directed by Satoshi Kon")

#result4 = retriever.batch(["Scientist","toys"])
#print(result4)

[Document(id='6b6f83ea-3723-4883-9b69-d5831a761463', metadata={'director': 'Satoshi Kon', 'rating': 8.6, 'year': 2006}, page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea')]

## Now let us see how to load a PDF into vector store and persist it.  This time we will use FAISS

 We will be using PyPdfLoader to load documents from pdf.

 There many document loaders available. See (https://api.python.langchain.com/en/latest/community_api_reference.html#module-langchain_community.document_loaders)

<font color="red"> **Before executing below code, travel.pdf  given to u**




In [15]:

from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter


#loader = PyPDFLoader(file_path="drive/MyDrive/Colab Notebooks/ds-book.pdf")
loader = PyPDFLoader(file_path="travel-policy.pdf")
documents = loader.load()


In [ ]:
documents[1]

**After all documents are loaded, we have to split them into chunks.**

**Observe the below code and understand how we are splitting documents in to chunks**

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50, separator="\n")
docs = text_splitter.split_documents(documents=documents)


**Observe the below code and understand how we are using **FAISS** to create vector store and save the documents**

**Execute the below code in a cell**

In [18]:
from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

#embeddings = OpenAIEmbeddings()

embeddings= HuggingFaceEmbeddings()

vectorstore = FAISS.from_documents(docs, embeddings)
vectorstore.save_local("faiss_store1")

**Creating a Retrieval Chain using Vector Store as retriever**

see the below code and understand how we are creating a retrieval chain using vector store as retirver

Execute the below code to  create a chain

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
from langchain.chains.retrieval import create_retrieval_chain

vectorstore = FAISS.load_local(
    "faiss_store1", embeddings=embeddings, allow_dangerous_deserialization=True
)

retriever = vectorstore.as_retriever(
    search_type="similarity",  # mmr or similarity_score_threshold
    search_kwargs={"k": 3},
)
message = """
        Answer this question using the provided context only.
        If the information is not available in the context, just reply with "i dont know"
        {input}
        Context:
        {context}
        """
prompt = ChatPromptTemplate.from_messages(
    [("human", message)],
)
llm = ChatOpenAI()
question_answer_chain = create_stuff_documents_chain(llm, prompt)
#print(question_answer_chain)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)
print(rag_chain)

# rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm
# rag_chain

**<font color="red"> Understand about create_map_reduce_documents_chain,create_refine_documents_chain,create_map_rerank_documents_chain.  Better u chat gpt to get summary of usecases,usage,etc**

**Invoking Retrieval chain**

Execute the below code to invoke the chain

In [ ]:
response = rag_chain.invoke({"input": "tell me about all the reimbursement policies"})
print(response)
print(response['answer'])
for  doc in response["context"]:
    print(doc.page_content)

# BUILDING AN APP SIMILAR TO CHATPDF

We want to build an app similar to chatpdf.
We need a function which will take a file, and store all the documents into vectorstore

Understand the below function and execute it

In [27]:
import tempfile
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

def load_pdf_into_vectorstore(file: tempfile) -> str:
    try:
        print("======Loading file==================")
        file_path = file.name
        loader = PyPDFLoader(file_path=file_path)
        documents = loader.load()
        text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=30, separator="\n")
        docs = text_splitter.split_documents(documents=documents)
        embeddings = HuggingFaceEmbeddings()

        #vectorstore = FAISS.from_documents(docs, embeddings)


        vectorstore = Chroma.from_documents(
            documents, embedding=embeddings , persist_directory="chromadb11"
        )
       # vectorstore.save_local("pdf_store")

        print("======File Loaded================== ")

        return 'Document uploaded and index created successfully. You can chat now.'
    except Exception as e:
        print(e)
        return e

**Now let us build an app using gradio which allows to upload pdf and chat**

in the below code, we are defining a function which will be invoked when user sends a message to chatbot.

Understand the code in the below cell and execute it

In [36]:
import gradio as gr
from langchain import OpenAI, PromptTemplate
from langchain.document_loaders import PyPDFLoader

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage


model = ChatOpenAI()


def getresponse(query, history:list) -> tuple:

   vectorstore = Chroma(
      persist_directory="chromadb11", embedding_function=embeddings
  )

 #  vectorstore = FAISS.load_local("pdf_store", embeddings= OpenAIEmbeddings(), allow_dangerous_deserialization=True)

   message = """
    Answer this question using the provided context . If information is not available in the context,
      Just respond saying "I dont know"
    {input}
    Context:
    {context}
    """

   prompt = ChatPromptTemplate.from_messages([("human", message)])
   llm = ChatOpenAI()


   question_answer_chain = create_stuff_documents_chain(llm, prompt)
#print(question_answer_chain)
   rag_chain = create_retrieval_chain(vectorstore.as_retriever(), question_answer_chain)

   # rag_chain = (
   #    {"context": vectorstore.as_retriever(), "question": RunnablePassthrough()}
   #    | prompt
   #    | llm
   # )



   response1= rag_chain.invoke({"input":query})
   print(response1)
   history.append((query, response1['answer']))
   return "",history


**Understand and execute the below code.**

**Once u go to the given url , upload a file and send queries and chat**

In [ ]:
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            file = gr.components.File(
                label='Upload your pdf file',
                file_count='single',
                file_types=['.pdf'])
            #with gr.Row():
            upload = gr.components.Button(
                    value='Upload', variant='primary')

        label = gr.components.Textbox()
    chatbot = gr.Chatbot(label='Talk to the Document')


    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])
    vectorStore =None

    upload.click(load_pdf_into_vectorstore,[file],[label])

    msg.submit(getresponse, [msg,chatbot], [msg, chatbot])

if __name__ == '__main__':
    demo.launch(debug=True)

**Let us try to understand how to retrieve data from a given web url**

In [ ]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load blog post
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data)

# VectorDB
embedding = HuggingFaceEmbeddings()
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)
vectordb

**Now Let us try to Understand MultiQueryRetriever**

The MultiQueryRetriever automates the process of prompt tuning by using an LLM to generate multiple queries from different perspectives for a given user input query. For each query, it retrieves a set of relevant documents and takes the unique union across all queries to get a larger set of potentially relevant documents.

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

question = "What are the approaches to Task Decomposition?"
llm = ChatOpenAI(temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm
)

In [ ]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [ ]:
unique_docs = retriever_from_llm.invoke(question)
unique_docs

**(Optional)See how to build  conversational rag at https://python.langchain.com/docs/how_to/qa_chat_history_how_to/**